In [1]:
%matplotlib inline

# Predicting positions of FIFA players

# Introduction

I introduce you to a dataset with all football players from one very popular PC game - FIFA. This dataset consists of almost 17 000 players from all around the world. Each row represents a particular player who has lots of information and statistics about him - like age, nationality, football club, prefered foot, etc. Those player "features' as we can name them are 88, so our dataset has 88 columns. We will concentrate precisely on the most interesting stats - the player skills like dribbling, finishing (scoring), passing, heading, marking etc. (their values are in some kind of range and the bigger number represents better qualification in the particular skill). On other hand, each player has its own role on the playfield and this role is more or less correlated with his skills. If he is a good defender - he will have better defending skills than attacking, if he plays in the middle of the field he will have a different set of skills. But the things are not that simple - there are so many different players with different capabilities so the mentioned differences are not that strict. There are also positions that are not that strictly separated and require both attacking, defending and other skills. A good example of such player positions are for example the defensive midfielder, the attacking midfielder, the wing back, etc. There are also players who have tons of skills and can play on each position on the field.

Here comes my role. I will try to predict each player's position based on the skills that he has. For this purpose I will use a set of algorithms and will try to find the best model that can predict a player's position.

In [2]:
import pandas as pd
import matplotlib.pyplot as plt

import seaborn as sns

from sklearn.model_selection import train_test_split

from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier


from sklearn.preprocessing import MinMaxScaler

from sklearn.metrics import classification_report

In [3]:
df = pd.read_csv('fifa2019-dataset.csv')


In [4]:
df.shape

(18207, 88)

### Display all the columns
As we can see - the columns are 88 and if we want to visualise them with head() or sample(), we won't be able to see all of them. To show all the columns of our dataframe and be able to decide which columns we will need, we must change the pandas display settings.

In [5]:
pd.set_option("display.max_columns", None)

In [6]:
df.head()

,ID,Name,Age,Photo,Nationality,Flag,Overall,Potential,Club,Club Logo,Value,Wage,Special,Preferred Foot,International Reputation,Weak Foot,Skill Moves,Work Rate,Body Type,Real Face,Position,Jersey Number,Joined,Loaned From,Contract Valid Until,Height,Weight,LS,ST,RS,LW,LF,CF,RF,RW,LAM,CAM,RAM,LM,LCM,CM,RCM,RM,LWB,LDM,CDM,RDM,RWB,LB,LCB,CB,RCB,RB,Crossing,Finishing,HeadingAccuracy,ShortPassing,Volleys,Dribbling,Curve,FKAccuracy,LongPassing,BallControl,Acceleration,SprintSpeed,Agility,Reactions,Balance,ShotPower,Jumping,Stamina,Strength,LongShots,Aggression,Interceptions,Positioning,Vision,Penalties,Composure,Marking,StandingTackle,SlidingTackle,GKDiving,GKHandling,GKKicking,GKPositioning,GKReflexes,Release Clause
0,158023,L. Messi,31,https://cdn.sofifa.org/players/4/19/158023.png,Argentina,https://cdn.sofifa.org/flags/52.png,94,94,FC Barcelona,https://cdn.sofifa.org/teams/2/light/241.png,€110.5M,€565K,2202,Left,5.0,4.0,4.0,Medium/ Medium,Messi,Yes,RF,10.0,1-Jul-04,NaN,2021,5'7,159lbs,88+2,88+2,88+2,92+2,93+2,93+2,93+2,92+2,93+2,93+2,93+2,91+2,84+2,84+2,84+2,91+2,64+2,61+2,61+2,61+2,64+2,59+2,47+2,47+2,47+2,59+2,84.0,95.0,70.0,90.0,86.0,97.0,93.0,94.0,87.0,96.0,91.0,86.0,91.0,95.0,95.0,85.0,68.0,72.0,59.0,94.0,48.0,22.0,94.0,94.0,75.0,96.0,33.0,28.0,26.0,6.0,11.0,15.0,14.0,8.0,€226.5M
1,20801,Cristiano Ronaldo,33,https://cdn.sofifa.org/players/4/19/20801.png,Portugal,https://cdn.sofifa.org/flags/38.png,94,94,Juventus,https://cdn.sofifa.org/teams/2/light/45.png,€77M,€405K,2228,Right,5.0,4.0,5.0,High/ Low,C. Ronaldo,Yes,ST,7.0,10-Jul-18,NaN,2022,6'2,183lbs,91+3,91+3,91+3,89+3,90+3,90+3,90+3,89+3,88+3,88+3,88+3,88+3,81+3,81+3,81+3,88+3,65+3,61+3,61+3,61+3,65+3,61+3,53+3,53+3,53+3,61+3,84.0,94.0,89.0,81.0,87.0,88.0,81.0,76.0,77.0,94.0,89.0,91.0,87.0,96.0,70.0,95.0,95.0,88.0,79.0,93.0,63.0,29.0,95.0,82.0,85.0,95.0,28.0,31.0,23.0,7.0,11.0,15.0,14.0,11.0,€127.1M
2,190871,Neymar Jr,26,https://cdn.sofifa.org/players/4/19/190871.png,Brazil,https://cdn.sofifa.org/flags/54.png,92,93,Paris Saint-Germain,https://cdn.sofifa.org/teams/2/light/73.png,€118.5M,€290K,2143,Right,5.0,5.0,5.0,High/ Medium,Neymar,Yes,LW,10.0,3-Aug-17,NaN,2022,5'9,150lbs,84+3,84+3,84+3,89+3,89+3,89+3,89+3,89+3,89+3,89+3,89+3,88+3,81+3,81+3,81+3,88+3,65+3,60+3,60+3,60+3,65+3,60+3,47+3,47+3,47+3,60+3,79.0,87.0,62.0,84.0,84.0,96.0,88.0,87.0,78.0,95.0,94.0,90.0,96.0,94.0,84.0,80.0,61.0,81.0,49.0,82.0,56.0,36.0,89.0,87.0,81.0,94.0,27.0,24.0,33.0,9.0,9.0,15.0,15.0,11.0,€228.1M
3,193080,De Gea,27,https://cdn.sofifa.org/players/4/19/193080.png,Spain,https://cdn.sofifa.org/flags/45.png,91,93,Manchester United,https://cdn.sofifa.org/teams/2/light/11.png,€72M,€260K,1471,Right,4.0,3.0,1.0,Medium/ Medium,Lean,Yes,GK,1.0,1-Jul-11,NaN,2020,6'4,168lbs,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,17.0,13.0,21.0,50.0,13.0,18.0,21.0,19.0,51.0,42.0,57.0,58.0,60.0,90.0,43.0,31.0,67.0,43.0,64.0,12.0,38.0,30.0,12.0,68.0,40.0,68.0,15.0,21.0,13.0,90.0,85.0,87.0,88.0,94.0,€138.6M
4,192985,K. De Bruyne,27,https://cdn.sofifa.org/players/4/19/192985.png,Belgium,https://cdn.sofifa.org/flags/7.png,91,92,Manchester City,https://cdn.sofifa.org/teams/2/light/10.png,€102M,€355K,2281,Right,4.0,5.0,4.0,High/ High,Normal,Yes,RCM,7.0,30-Aug-15,NaN,2023,5'11,154lbs,82+3,82+3,82+3,87+3,87+3,87+3,87+3,87+3,88+3,88+3,88+3,88+3,87+3,87+3,87+3,88+3,77+3,77+3,77+3,77+3,77+3,73+3,66+3,66+3,66+3,73+3,93.0,82.0,55.0,92.0,82.0,86.0,85.0,83.0,91.0,91.0,78.0,76.0,79.0,91.0,77.0,91.0,63.0,90.0,75.0,91.0,76.0,61.0,87.0,94.0,79.0,88.0,68.0,58.0,51.0,15.0,13.0,5.0,10.0,13.0,€196.4M


# Data Preparation

It seems that the columns that we are interested in are **Poisition** - this will be our target field, and the skills of the players which are starting with the column **Crossing** all the way up to the last field excluding **Release Clause**. Those skills are with some values from low to high and the higher number - more skillfull the player is. Lets create two dataframes - one with the positions (our target) and one with the skills we want to track and measure.

In [7]:
df.Position

0         RF
1         ST
2         LW
3         GK
4        RCM
        ... 
18202     CM
18203     ST
18204     ST
18205     RW
18206     CM
Name: Position, Length: 18207, dtype: object

In [8]:
df.groupby(['Position']).size()

Position
CAM     958
CB     1778
CDM     948
CF       74
CM     1394
GK     2025
LAM      21
LB     1322
LCB     648
LCM     395
LDM     243
LF       15
LM     1095
LS      207
LW      381
LWB      78
RAM      21
RB     1291
RCB     662
RCM     391
RDM     248
RF       16
RM     1124
RS      203
RW      370
RWB      87
ST     2152
dtype: int64

In [9]:
df.Position.unique()

array(['RF', 'ST', 'LW', 'GK', 'RCM', 'LF', 'RS', 'RCB', 'LCM', 'CB',
       'LDM', 'CAM', 'CDM', 'LS', 'LCB', 'RM', 'LAM', 'LM', 'LB', 'RDM',
       'RW', 'CM', 'RB', 'RAM', 'CF', 'RWB', 'LWB', nan], dtype=object)

Seems like we have players that don't have a valid position. Lets see how many such players we have.

In [10]:
df.Position.isnull().sum()

60

We can drop those 60 players from our dataframe.

In [11]:
df = df.dropna(subset=['Position'])

In [12]:
df.Position.isnull().sum()

0

Our players have a lot of different positions. This complicates our task as there is no big difference int he skills needed for player who is LM (left middlefielder), LCM (left central middfielder) and LAM (left attacking miffielder), etc. So, lets do some generalozation and group the player positions into 4 different playrolles, rather than playing positions. We can group them into Goalkeepers, Defenders, Midfielders and Strikers and just try to predict if a player plays into one of those 4 major groups.

**So, our goal will be to detetmine if a player plays into each of those 4 groups.**

- Strikers = RF, CF, LF, RS, LS, ST
- Midfielders = LW, LAM, LM, LCM, LDM, CAM, CM, CDM, RDM, RCM, RM, RAM, RW
- Defenders = LWB, LB, LCB, CB, RCB, RB, RWB
- Goalkeepers = GK

In [13]:
df.Position = df.Position.replace(['RF', 'CF', 'LF', 'RS', 'LS', 'ST'], "FORWARD")
df.Position = df.Position.replace(['LW', 'LAM', 'LM', 'LCM', 'LDM', 'CAM', 'CM', 'CDM', 'RDM', 'RCM', 'RM', 'RAM', 'RW'], "MIDFIELDER")
df.Position = df.Position.replace(['LWB', 'LB', 'LCB', 'CB', 'RCB', 'RB', 'RWB'], "DEFENDER")
df.Position.replace('GK', "GOALKEEPER")

0           FORWARD
1           FORWARD
2        MIDFIELDER
3        GOALKEEPER
4        MIDFIELDER
            ...    
18202    MIDFIELDER
18203       FORWARD
18204       FORWARD
18205    MIDFIELDER
18206    MIDFIELDER
Name: Position, Length: 18147, dtype: object

In [14]:
df.Position.unique()

array(['FORWARD', 'MIDFIELDER', 'GK', 'DEFENDER'], dtype=object)

Great! We have grouped all positions into 4 different playing roles but strangly, there are some players with a position **"nan"**. Lets see what are those players and decide what to do. 

In [15]:
df.groupby(['Position']).size()


Position
DEFENDER      5866
FORWARD       2667
GK            2025
MIDFIELDER    7589
dtype: int64

Now our column with the player **Positions** looks OK. We have 4 different types of player roles and our logarithm will need to categorise each player to fit into those 4 types. 

In [16]:
positions = df.Position
positions

0           FORWARD
1           FORWARD
2        MIDFIELDER
3                GK
4        MIDFIELDER
            ...    
18202    MIDFIELDER
18203       FORWARD
18204       FORWARD
18205    MIDFIELDER
18206    MIDFIELDER
Name: Position, Length: 18147, dtype: object

We are interested in the columns from Crossing to the end. Lets find the id of the Crossing column and create a dataframe for all the skills and call it **skills**.

In [17]:
df.columns.get_loc("Crossing")

53

In [18]:
skills = df.iloc[:, 53 : -1]
skills.shape

(18147, 34)

In [19]:
skills.sample(5)

,Crossing,Finishing,HeadingAccuracy,ShortPassing,Volleys,Dribbling,Curve,FKAccuracy,LongPassing,BallControl,Acceleration,SprintSpeed,Agility,Reactions,Balance,ShotPower,Jumping,Stamina,Strength,LongShots,Aggression,Interceptions,Positioning,Vision,Penalties,Composure,Marking,StandingTackle,SlidingTackle,GKDiving,GKHandling,GKKicking,GKPositioning,GKReflexes
7906,60.0,67.0,56.0,63.0,62.0,69.0,65.0,34.0,60.0,66.0,77.0,77.0,78.0,63.0,64.0,65.0,63.0,66.0,65.0,66.0,46.0,42.0,64.0,60.0,46.0,58.0,27.0,27.0,30.0,12.0,14.0,12.0,8.0,6.0
14323,58.0,60.0,44.0,63.0,49.0,66.0,45.0,39.0,51.0,60.0,68.0,65.0,50.0,58.0,55.0,66.0,40.0,46.0,65.0,40.0,32.0,21.0,60.0,45.0,60.0,51.0,35.0,26.0,27.0,11.0,14.0,13.0,6.0,13.0
16186,23.0,60.0,63.0,45.0,43.0,52.0,31.0,31.0,29.0,55.0,51.0,57.0,50.0,56.0,48.0,59.0,62.0,56.0,76.0,49.0,66.0,18.0,58.0,41.0,54.0,55.0,20.0,25.0,18.0,13.0,6.0,5.0,14.0,6.0
7625,45.0,20.0,65.0,57.0,18.0,36.0,31.0,35.0,57.0,49.0,33.0,36.0,34.0,55.0,55.0,65.0,83.0,57.0,89.0,32.0,65.0,66.0,29.0,29.0,46.0,66.0,70.0,67.0,66.0,5.0,11.0,6.0,5.0,10.0
12251,62.0,47.0,56.0,60.0,48.0,58.0,57.0,48.0,59.0,53.0,71.0,79.0,71.0,59.0,68.0,60.0,66.0,86.0,64.0,59.0,66.0,59.0,55.0,53.0,58.0,55.0,47.0,58.0,59.0,12.0,13.0,14.0,15.0,14.0


In [20]:
skills.describe()

,Crossing,Finishing,HeadingAccuracy,ShortPassing,Volleys,Dribbling,Curve,FKAccuracy,LongPassing,BallControl,Acceleration,SprintSpeed,Agility,Reactions,Balance,ShotPower,Jumping,Stamina,Strength,LongShots,Aggression,Interceptions,Positioning,Vision,Penalties,Composure,Marking,StandingTackle,SlidingTackle,GKDiving,GKHandling,GKKicking,GKPositioning,GKReflexes
count,18147.000000,18147.000000,18147.000000,18147.000000,18147.000000,18147.000000,18147.000000,18147.000000,18147.000000,18147.000000,18147.000000,18147.000000,18147.000000,18147.000000,18147.000000,18147.000000,18147.000000,18147.000000,18147.000000,18147.000000,18147.000000,18147.000000,18147.000000,18147.000000,18147.000000,18147.000000,18147.000000,18147.000000,18147.000000,18147.000000,18147.000000,18147.000000,18147.000000,18147.000000
mean,49.738414,45.550229,52.300766,58.695432,42.912217,55.375158,47.176283,42.866038,52.721386,58.374828,64.612829,64.726236,63.501295,61.839147,63.964292,55.465201,65.091034,63.221579,65.318620,47.113187,55.876068,46.702761,49.962198,53.407781,48.546371,58.651127,47.286053,47.701879,45.666336,16.616906,16.393839,16.233041,16.389651,16.712019
std,18.364255,19.527445,17.381753,14.696075,17.695900,18.912224,18.396009,17.480034,15.325211,16.685643,14.930320,14.651776,14.768956,9.011056,14.136073,17.235534,11.822327,15.896381,12.552479,19.263142,17.366534,20.697462,19.530469,14.146594,15.703113,11.437138,19.900450,21.663630,21.287961,17.698612,16.909971,16.504103,17.037031,17.957521
min,5.000000,2.000000,4.000000,7.000000,4.000000,4.000000,6.000000,3.000000,9.000000,5.000000,12.000000,12.000000,14.000000,21.000000,16.000000,2.000000,15.000000,12.000000,17.000000,3.000000,11.000000,3.000000,2.000000,10.000000,5.000000,3.000000,3.000000,2.000000,3.000000,1.000000,1.000000,1.000000,1.000000,1.000000
25%,38.000000,30.000000,44.000000,54.000000,30.000000,49.000000,34.000000,31.000000,43.000000,54.000000,57.000000,57.000000,55.000000,56.000000,56.000000,45.000000,58.000000,56.000000,58.000000,33.000000,44.000000,26.000000,38.000000,44.000000,39.000000,51.000000,30.000000,27.000000,24.000000,8.000000,8.000000,8.000000,8.000000,8.000000
50%,54.000000,49.000000,56.000000,62.000000,44.000000,61.000000,48.000000,41.000000,56.000000,63.000000,67.000000,67.000000,66.000000,62.000000,66.000000,59.000000,66.000000,66.000000,67.000000,51.000000,59.000000,52.000000,55.000000,55.000000,49.000000,60.000000,53.000000,55.000000,52.000000,11.000000,11.000000,11.000000,11.000000,11.000000
75%,64.000000,62.000000,64.000000,68.000000,57.000000,68.000000,62.000000,57.000000,64.000000,69.000000,75.000000,75.000000,74.000000,68.000000,74.000000,68.000000,73.000000,74.000000,74.000000,62.000000,69.000000,64.000000,64.000000,64.000000,60.000000,67.000000,64.000000,66.000000,64.000000,14.000000,14.000000,14.000000,14.000000,14.000000
max,93.000000,95.000000,94.000000,93.000000,90.000000,97.000000,94.000000,94.000000,93.000000,96.000000,97.000000,96.000000,96.000000,96.000000,96.000000,95.000000,95.000000,96.000000,97.000000,94.000000,95.000000,92.000000,95.000000,94.000000,92.000000,96.000000,94.000000,93.000000,91.000000,90.000000,92.000000,91.000000,90.000000,94.000000


### Normalization of the skill values

It looks like the skills are somwhere in the range between 1 and 97 so it will be a good idea to use normalization and fit them between 0 and 1. 

In [21]:
scaler = MinMaxScaler()
scaler.fit(skills)

MinMaxScaler()

In [22]:
skills_scaled = scaler.transform(skills)
skills_scaled

array([[0.89772727, 1.        , 0.73333333, ..., 0.15555556, 0.14606742,
        0.07526882],
       [0.89772727, 0.98924731, 0.94444444, ..., 0.15555556, 0.14606742,
        0.10752688],
       [0.84090909, 0.91397849, 0.64444444, ..., 0.15555556, 0.15730337,
        0.10752688],
       ...,
       [0.22727273, 0.40860215, 0.46666667, ..., 0.1       , 0.05617978,
        0.12903226],
       [0.44318182, 0.51612903, 0.38888889, ..., 0.14444444, 0.07865169,
        0.08602151],
       [0.40909091, 0.34408602, 0.46666667, ..., 0.08888889, 0.12359551,
        0.08602151]])

In [23]:
skills_train, skills_test, position_train, position_test = train_test_split(skills_scaled, positions, test_size = 0.20)

In [24]:
skills_train.shape, position_train.shape

((14517, 34), (14517,))

In [25]:
skills_test.shape, position_test.shape

((3630, 34), (3630,))

In [26]:
logistic = LogisticRegression(max_iter=10000)

In [27]:
logistic.fit(skills_train, position_train)

LogisticRegression(max_iter=10000)

In [28]:
logistic.score(skills_test, position_test)

0.9027548209366392

In [29]:
print(classification_report(position_train, logistic.predict(skills_train)))

              precision    recall  f1-score   support

    DEFENDER       0.92      0.92      0.92      4651
     FORWARD       0.88      0.85      0.86      2178
          GK       1.00      1.00      1.00      1612
  MIDFIELDER       0.88      0.90      0.89      6076

    accuracy                           0.91     14517
   macro avg       0.92      0.92      0.92     14517
weighted avg       0.91      0.91      0.91     14517



In [30]:
lr_params = {
    "C" : [0.01, 0.1, 1, 10, 100, 1000]
}

In [31]:
grid_search = GridSearchCV(LogisticRegression(max_iter=10000), param_grid=lr_params)

In [32]:
grid_search.fit(skills_test, position_test)

GridSearchCV(estimator=LogisticRegression(max_iter=10000),
             param_grid={'C': [0.01, 0.1, 1, 10, 100, 1000]})

In [33]:
grid_search.score(skills_test, position_test)

0.9027548209366392

In [34]:
print(classification_report(position_train, grid_search.predict(skills_train)))

              precision    recall  f1-score   support

    DEFENDER       0.91      0.92      0.92      4651
     FORWARD       0.87      0.84      0.86      2178
          GK       1.00      1.00      1.00      1612
  MIDFIELDER       0.89      0.89      0.89      6076

    accuracy                           0.90     14517
   macro avg       0.92      0.91      0.92     14517
weighted avg       0.90      0.90      0.90     14517



In [35]:
grid_search.best_estimator_

LogisticRegression(C=10, max_iter=10000)

In [36]:
grid_search.best_params_

{'C': 10}

In [37]:
grid_search.cv_results_

{'mean_fit_time': array([0.09605021, 0.12881494, 0.25653577, 0.50444832, 1.06781225,
        0.65314574]),
 'std_fit_time': array([0.04337899, 0.01439245, 0.03783225, 0.05077619, 0.11865044,
        0.18048482]),
 'mean_score_time': array([0.00080204, 0.00026679, 0.00170608, 0.00024219, 0.00039024,
        0.00161152]),
 'std_score_time': array([0.00075482, 0.00053358, 0.00293761, 0.00048437, 0.00078049,
        0.00322304]),
 'param_C': masked_array(data=[0.01, 0.1, 1, 10, 100, 1000],
              mask=[False, False, False, False, False, False],
        fill_value='?',
             dtype=object),
 'params': [{'C': 0.01},
  {'C': 0.1},
  {'C': 1},
  {'C': 10},
  {'C': 100},
  {'C': 1000}],
 'split0_test_score': array([0.79752066, 0.90358127, 0.90633609, 0.91184573, 0.90633609,
        0.90633609]),
 'split1_test_score': array([0.7630854 , 0.84710744, 0.87465565, 0.8815427 , 0.88016529,
        0.88016529]),
 'split2_test_score': array([0.77134986, 0.85123967, 0.87878788, 0.87741047, 0

In [38]:
decision_tree = DecisionTreeClassifier()

In [39]:
decision_tree.fit(skills_train, position_train)

DecisionTreeClassifier()

In [40]:
dec_tree = decision_tree.score(skills_test, position_test)
dec_tree

0.8325068870523415

In [41]:
random_forest = RandomForestClassifier()

In [42]:
random_forest.fit(skills_train, position_train)

RandomForestClassifier()

In [43]:
rand_forest = random_forest.score(skills_test, position_test)
rand_forest

0.9085399449035813

In [44]:
print(classification_report(position_train, random_forest.predict(skills_train)))

              precision    recall  f1-score   support

    DEFENDER       1.00      1.00      1.00      4651
     FORWARD       1.00      1.00      1.00      2178
          GK       1.00      1.00      1.00      1612
  MIDFIELDER       1.00      1.00      1.00      6076

    accuracy                           1.00     14517
   macro avg       1.00      1.00      1.00     14517
weighted avg       1.00      1.00      1.00     14517



In [45]:
random_forest.feature_importances_

array([0.04313883, 0.06929684, 0.06520637, 0.02538519, 0.01880392,
       0.02666025, 0.02260157, 0.02413412, 0.04080425, 0.01504551,
       0.01220883, 0.01444534, 0.00981314, 0.00815914, 0.0131692 ,
       0.01582081, 0.01371706, 0.00928249, 0.02137015, 0.03588498,
       0.01085779, 0.0372721 , 0.02516922, 0.05350802, 0.01872001,
       0.00803141, 0.03561125, 0.06361383, 0.08862959, 0.03596012,
       0.03165752, 0.02174408, 0.02334722, 0.04092983])

In [46]:
ada = AdaBoostClassifier()

In [47]:
ada.fit(skills_train, position_train)

AdaBoostClassifier()

In [48]:
ada.score(skills_test, position_test)

0.5672176308539945

In [49]:
print(classification_report(position_train, ada.predict(skills_train)))

              precision    recall  f1-score   support

    DEFENDER       0.87      0.47      0.61      4651
     FORWARD       0.35      0.96      0.51      2178
          GK       1.00      1.00      1.00      1612
  MIDFIELDER       0.53      0.38      0.45      6076

    accuracy                           0.57     14517
   macro avg       0.69      0.70      0.64     14517
weighted avg       0.66      0.57      0.57     14517



In [50]:
ada.feature_importances_

array([0.  , 0.22, 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  ,
       0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  ,
       0.18, 0.02, 0.  , 0.02, 0.02, 0.  , 0.04, 0.22, 0.08, 0.  , 0.14,
       0.06])